In [1]:
import numpy as np
import torch
from torchvision import transforms
from tifffile import imread
import sys
sys.path.append('../')
from noise_model.PixelCNN import PixelCNN
import os
from utils.dataloaders import create_nm_loader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Load noise samples

In [3]:
noise_location = '../data/conv/noise.tif'
noise = imread(noise_location)

Create data loaders and get the mean and standard deviation of the noise samples

In [4]:
transform = None
nm_train_loader, nm_val_loader, noise_mean, noise_std = create_nm_loader(noise, batch_size=8, split=0.8, transform=transform)

Set noise model checkpoint directory

In [5]:
nm_checkpoint_path = '../nm_checkpoint/conv'
if not os.path.exists(nm_checkpoint_path):
    os.makedirs(nm_checkpoint_path)

Initialise trainer and noise model

In [6]:
trainer = pl.Trainer(default_root_dir=nm_checkpoint_path,
                     accelerator='gpu',
                     devices=1,
                     max_epochs=500,
                     logger=TensorBoardLogger(nm_checkpoint_path),
                     callbacks=[ModelCheckpoint(save_weights_only=False, mode="min", monitor="val_nll"),
                                LearningRateMonitor(logging_interval='epoch'),
                                EarlyStopping(patience=10, monitor='val_nll')])

noise_model = PixelCNN(in_channels=1, 
                       n_gaussians=10,
                       n_filters=128,
                       kernel_size=7,
                       depth=5,
                       dropout=0,
                       noise_mean=noise_mean,
                       noise_std=noise_std).to(device)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Train and save final parameters

In [7]:
trainer.fit(noise_model, nm_train_loader, nm_val_loader)
trainer.save_checkpoint(os.path.join(nm_checkpoint_path, 'final_params.ckpt'))

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                  | Params
------------------------------------------------------
0 | up_inconv   | VerticalConvolution   | 6.4 K 
1 | left_inconv | HorizontalConvolution | 1.0 K 
2 | dropout     | Dropout               | 0     
3 | gatedconvs  | ModuleList            | 1.9 M 
4 | outconv     | Conv2d                | 3.9 K 
------------------------------------------------------
1.9 M     Trainable params
0         Non-trainable params
1.9 M     Total params
7.716     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/ben/miniconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 36 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/ben/miniconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 36 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/ben/miniconda3/envs/pl/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1933: PossibleUserWarning: The number of training batches (6) is smaller than the logging in

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]